In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import library

In [3]:
import numpy as np
import math
# from . import calc_util as cu
import sys
sys.path.append('src')
import dhw_ex_model as dem
import dartwork_mpl as dm
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.ticker as ticker
dm.use_dmpl_style()


Load colors...
Load colormaps...


# 0. 준비

In [4]:
## Fontsize 지정
plt.rcParams['font.size'] = 9

fs = {
    'label': dm.fs(0),
    'tick': dm.fs(-1.5),
    'legend': dm.fs(-2.0),
    'subtitle': dm.fs(-0.5),
    'cbar_tick': dm.fs(-2.0),
    'cbar_label': dm.fs(-2.0),
    'cbar_title': dm.fs(-1),
    'setpoint': dm.fs(-1),
    'text': dm.fs(-3.0),
            }

pad = {
    'label': 6,
    'tick': 5,
}

LW = np.arange(0.25, 3.0, 0.25)

## 1.1 Constant

In [5]:
c_w = 4186  # J/kgK
rho_w = 1000  # kg/m3

# 2. Function

In [6]:
# =============================================================================
# 5) (옵션) 플롯 시작부
#    - 한글 폰트 설정 등은 사용 OS 환경에 맞게 수동 설정
# =============================================================================
def plot_simple_graph(df_column, time, xlabel, ylabel, xmin = 0, xmax=24, Kelvin=False, color = 'dm.blue5', savepath = None):
    # 온도 데이터를 섭씨(°C)로 변환하여 플로팅
    if Kelvin:
        y = dem.K2C(df_column)
    else:
        y = df_column
    x = time * dem.s2h  # 초를 시간으로 변환

    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    
    # --- 마이너 틱 설정 ---

    # X축: 주 틱 사이를 5개의 간격으로 나눔 (마이너 틱 4개 생성)
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    # Y축: 주 틱 사이를 2개의 간격으로 나눔 (마이너 틱 1개 생성)
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    ax.plot(x, y, color=color, linewidth=0.5,)
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))

    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    ax.set_ylim(np.nanmin(y)*0.9, np.nanmax(y)*1.1)
    # ax.set_ylim(50, 55)
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox = [0, 1, 0.02, 1])
    plt.savefig(f'{savepath}.png')
    dm.save_and_show(fig,)
    plt.close()

def plot_multi_graph(df_columns, legends, time, xlabel, ylabel, linestyles=None, colors=None, xmin=0, xmax=24, Kelvin=False, savepath=None, scatter = False):
    """
    여러 개의 데이터 열을 받아 하나의 그래프에 플로팅하는 함수.
    scatter=True이면 점 그래프(scatter plot)를 그립니다.

    Args:
        *df_columns: 플로팅할 데이터 열들 (예: df['col1'], df['col2'], ...)
        legends (list or tuple): 각 데이터 열에 해당하는 레이블 리스트
        time (array-like): x축 시간 데이터
        ... (기존 인자들과 동일)
        colors (list or tuple, optional): 각 플롯에 사용할 색상 리스트. 기본값 None.
        scatter (bool): True이면 scatter plot, False이면 line plot을 그립니다.
    """
    if len(df_columns) != len(legends):
        raise ValueError("데이터 컬럼의 개수와 레이블의 개수가 일치해야 합니다.")

    # --- 1. 그래프 기본 설정 ---
    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    x = time * dem.s2h  # 초를 시간으로 변환
    
    # 기본 색상 리스트 (colors 인자가 주어지지 않을 경우 사용)
    if colors is None:
        colors = ['dm.blue5', 'dm.orange5', 'dm.green5', 'dm.red5', 'dm.violet5',
                  'dm.gray5', 'dm.yellow5', 'dm.cyan5']
    
    # --- 2. 모든 데이터를 바탕으로 Y축 min/max 계산 ---
    global_min = np.inf
    global_max = -np.inf
    
    processed_ys = []
    for col in df_columns:
        y = dem.K2C(col) if Kelvin else col
        processed_ys.append(y)
        if not y.empty:
            global_min = min(global_min, np.nanmin(y))
            global_max = max(global_max, np.nanmax(y))
            
    # --- 3. 반복문을 통해 여러 데이터 플로팅 (수정된 부분) ---
    for i, y_data in enumerate(processed_ys):
        
        current_color = colors[i % len(colors)]
        current_label = legends[i]

        if scatter:
            # scatter=True이면 ax.scatter() 호출
            ax.scatter(x, y_data, 
                       color=current_color,
                       label=current_label,
                       s=1.5, # 점 크기 (필요에 따라 조절)
                       alpha=0.3 # 투명도 (점이 겹칠 경우 유용)
                      )
        else:
            # scatter=False이면 기존 ax.plot() 호출
            current_linestyle = linestyles[i % len(linestyles)] if linestyles is not None else ['-', '--', '-.', ':'][i % 4]
            ax.plot(x, y_data, 
                    color=current_color,
                    linewidth=0.8, 
                    label=current_label,
                    linestyle=current_linestyle,
                   )

    # --- 4. 축 및 레이아웃 설정 ---
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))
    
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    
    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    global_int = global_max - global_min
    
    # Y축 범위 설정 (무한대 값 방지)
    if np.isinf(global_min) or np.isinf(global_max):
        ax.set_ylim(0, 1) # 기본값 설정 또는 다른 적절한 처리
    else:
        ax.set_ylim(global_min - global_int * 0.1, global_max + global_int * 0.3) # 전체 데이터 기준 ylim 설정
    
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])

    ax.legend(ncol = 6, fontsize = fs['legend']) # 범례 표시
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=[0, 1, 0.02, 1])
    
    if savepath:
        plt.savefig(f'{savepath}.png')
        
    dm.save_and_show(fig)
    plt.close()

# 3. 시뮬레이션 설정

# 4. Simulation 실행 및 csv 저장

In [7]:
# --- 1. 시뮬레이션 시간 및 태양 복사 배열 생성 ---
duration_hours = 24
steps_per_hour = 60 # 1분 간격
total_steps = duration_hours * steps_per_hour

# 시간 배열 생성 (단위: 시간 [h])
time_hours = np.linspace(0, duration_hours, total_steps, endpoint=False)
# 플로팅 함수용 시간 배열 (단위: 초 [s])
time_sec = time_hours * 3600

# 태양 복사 프로필 생성 (정오(12시)에 피크를 갖는 코사인파)
# np.cos((t - 12) * 2 * pi / 24) -> t=12일때 1, t=6, 18일때 0, t=0, 24일때 -1
peak_I_DN = 600.0  # W/m² (직달 최대값)
peak_I_dH = 120.0  # W/m² (산란 최대값)

# 코사인 프로필 (음수는 0으로)
radiation_profile1 = np.maximum(0, np.cos((time_hours - 12) * (2 * np.pi) / 24))
radiation_profile2 = np.maximum(0, np.cos((time_hours - 12) * (2 * np.pi) / 24) + 0.1)

I_DN_array = radiation_profile1 * peak_I_DN
I_dH_array = radiation_profile2 * peak_I_dH

# --- 2. 시뮬레이션 루프 실행 ---

# PV 시스템 인스턴스 생성 (설정값은 __post_init__에서 로드됨)
system = dem.PVSystemAnalyzer()

results_list = [] # 결과를 저장할 리스트

print(f"총 {total_steps} 스텝 (1분 간격) 시뮬레이션 진행...")

for i in range(total_steps):
    # 1. 현재 시간의 태양 복사 값으로 시스템 업데이트
    system.I_DN = I_DN_array[i]
    system.I_dH = I_dH_array[i]
    
    # 2. 시스템 에너지/엔트로피/엑서지 계산 실행
    system.system_update()
    
    # 3. 결과 저장
    # vars(system)은 객체의 모든 속성을 딕셔너리로 반환
    # .copy()를 사용하여 현재 상태를 복사해 리스트에 추가
    results_list.append(vars(system).copy())
    
print("...시뮬레이션 완료.")

총 1440 스텝 (1분 간격) 시뮬레이션 진행...
...시뮬레이션 완료.


# 4. Visualization

In [8]:
results_df = pd.DataFrame(results_list)
results_df.to_csv('PV_system_simulation_results.csv', index=False)

In [9]:
df = pd.read_csv('PV_system_simulation_results.csv')

In [10]:
df['Q_l_ctrl']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1435    0.0
1436    0.0
1437    0.0
1438    0.0
1439    0.0
Name: Q_l_ctrl, Length: 1440, dtype: float64

In [13]:
# # --- 3. 결과를 Pandas DataFrame으로 변환 ---
# # 플로팅 함수가 DataFrame 컬럼을 입력으로 받기 용이함


# # --- 4. 시각화 ---
# print("그래프 플로팅 시작...")

# # 그래프 1: 태양 복사량
# plot_multi_graph(
#     df_columns=[df['I_DN'], df['I_dH'], df['I_sol']],
#     legends=['Direct ($I_{DN}$)', 'Diffuse ($I_{dH}$)', 'Total ($I_{sol}$)'],
#     time=time_sec,
#     xlabel='Elapsed time [h]',
#     ylabel='Solar irradiance [W/$m^2$]',
#     savepath='plot_01_radiation'
# )

# # 그래프 2: 주요 온도 변화 (T_pv, T0)
# # T0는 켈빈(K)이지만 상수이므로 Kelvin=True 플래그와 함께 전달
# plot_multi_graph(
#     df_columns=[df['T_pv'], df['T0']],
#     legends=['PV Panel ($T_{pv}$)', 'Ambient ($T_0$)'],
#     time=time_sec,
#     xlabel='Elapsed time [h]',
#     ylabel='Temperature [°C]',
#     Kelvin=True, # Y축 값을 섭씨로 변환
#     savepath='plot_02_temperature'
# )

# # 그래프 3: 에너지(전력) 흐름
# plot_multi_graph(
#     df_columns=[df['E_pv0'], df['E_pv1'], df['E_pv2'], df['E_pv3']],
#     legends=['$E_{pv0}$ (Panel)', '$E_{pv1}$ (Controller)', '$E_{pv2}$ (Battery)', '$E_{pv3}$ (AC Output)'],
#     time=time_sec,
#     xlabel='Elapsed time [h]',
#     ylabel='Electric power [W]',
#     savepath='plot_03_energy_flow'
# )

# # 그래프 4: 엑서지 파괴(손실)
# plot_multi_graph(
#     df_columns=[df['X_c_pv'], df['X_c_ctrl'], df['X_c_batt'], df['X_c_DC_AC']],
#     legends=['$X_{c,pv}$ (Panel)', '$X_{c,ctrl}$ (Controller)', '$X_{c,batt}$ (Battery)', '$X_{c,DC\_AC}$ (Converter)'],
#     time=time_sec,
#     xlabel='Elapsed time [h]',
#     ylabel='Exergy consumption rate [W]',
#     savepath='plot_04_exergy_consumption_rate'
# )

# plot_multi_graph(
#     df_columns=[df['X_c_pv'], df['X_c_ctrl'], df['X_c_batt'], df['X_c_DC_AC']],
#     legends=['$X_{c,pv}$ (Panel)', '$X_{c,ctrl}$ (Controller)', '$X_{c,batt}$ (Battery)', '$X_{c,DC\_AC}$ (Converter)'],
#     time=time_sec,
#     xlabel='Elapsed time [h]',
#     ylabel='Exergy consumption rate [W]',
#     savepath='plot_04_exergy_consumption_rate'
# )

# plot_multi_graph(
#     df_columns=[df['X_c_ctrl'], df['X_c_batt'], df['X_c_DC_AC']],
#     legends=['$X_{c,ctrl}$ (Controller)', '$X_{c,batt}$ (Battery)', '$X_{c,DC\_AC}$ (Converter)'],
#     time=time_sec,
#     colors=['dm.orange5', 'dm.green5', 'dm.red5'],
#     linestyles=['--', '-.', ':'],
#     xlabel='Elapsed time [h]',
#     ylabel='Exergy consumption rate [W]',
#     savepath='plot_04_exergy_consumption_rate2'
# )


# plot_multi_graph(
#     df_columns=[df['Q_l_pv'], df['Q_l_ctrl'], df['Q_l_batt'], df['Q_l_DC_AC']],
#     legends=['$Q_{l,pv}$ (PV)', '$Q_{l,ctrl}$ (Controller)', '$Q_{l,batt}$ (Battery)', '$Q_{l,DC\_AC}$ (Converter)'],
#     time=time_sec,
#     colors=['dm.blue5','dm.orange5', 'dm.green5', 'dm.red5'],
#     linestyles=['-','--', '-.', ':'],
#     xlabel='Elapsed time [h]',
#     ylabel='Heat loss rate [W]',
#     savepath='plot_05_heat_loss_rate'
# )

# plot_multi_graph(
#     df_columns=[df['Q_l_ctrl'], df['Q_l_batt'], df['Q_l_DC_AC']],
#     legends=['$Q_{l,ctrl}$ (Controller)', '$Q_{l,batt}$ (Battery)', '$Q_{l,DC\_AC}$ (Converter)'],
#     time=time_sec,
#     colors=['dm.orange5', 'dm.green5', 'dm.red5'],
#     linestyles=['--', '-.', ':'],
#     xlabel='Elapsed time [h]',
#     ylabel='Heat loss rate [W]',
#     savepath='plot_05_heat_loss_rate2'
# )

plot_multi_graph(
    df_columns=[df['X_l_pv'], df['X_l_ctrl'], df['X_l_batt'], df['X_l_DC_AC']],
    legends=['$X_{l,pv}$ (PV)', '$X_{l,ctrl}$ (Controller)', '$X_{l,batt}$ (Battery)', '$X_{l,DC\_AC}$ (Converter)'],
    time=time_sec,
    colors=['dm.blue5','dm.orange5', 'dm.green5', 'dm.red5'],
    linestyles=['-','--', '-.', ':'],
    xlabel='Elapsed time [h]',
    ylabel='Exergy loss rate [W]',
    savepath='plot_06_exergy_loss_rate'
)

plot_multi_graph(
    df_columns=[df['X_l_ctrl'], df['X_l_batt'], df['X_l_DC_AC']],
    legends=['$X_{l,ctrl}$ (Controller)', '$X_{l,batt}$ (Battery)', '$X_{l,DC\_AC}$ (Converter)'],
    time=time_sec,
    colors=['dm.orange5', 'dm.green5', 'dm.red5'],
    linestyles=['--', '-.', ':'],
    xlabel='Elapsed time [h]',
    ylabel='Exergy loss rate [W]',
    savepath='plot_06_exergy_loss_rate2'
)

# # 그래프 5: (예시) T_pv만 단일 그래프로
# plot_simple_graph(
#     df_column=df['T_pv'],
#     time=time_sec,
#     xlabel='Elapsed time [h]',
#     ylabel='PV panel temperature [°C]',
#     Kelvin=True, # 섭씨로 변환
#     color='dm.red5',
#     savepath='plot_05_Tpv_simple'
# )



print("--- 모든 작업 완료 ---")

--- 모든 작업 완료 ---
